In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import mobilenet_v2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import os

In [2]:
import cv2
import tensorflow as tf


In [4]:


# Load the trained mask detection model
model = tf.keras.models.load_model("/content/mask_detector.model")

# Initialize the video stream
vs = cv2.VideoCapture(0)  # 0 for the default camera, you can change this if you have multiple cameras

while True:
    # Read a frame from the video stream
    ret, frame = vs.read()

    if not ret:
        break

    # Resize the frame to match the input size of your model (224x224)
    frame = cv2.resize(frame, (224, 224))

    # Preprocess the frame for prediction
    frame = preprocess_input(frame)

    # Expand the dimensions of the frame to match the model's input shape
    frame = np.expand_dims(frame, axis=0)

    # Perform inference to predict whether the person is wearing a mask or not
    predictions = model.predict(frame)

    # Get the label with the highest probability (0 for 'with mask', 1 for 'without mask')
    label = np.argmax(predictions, axis=1)[0]

    # Define the label and color for the bounding box
    if label == 0:
        label_text = "With Mask"
        color = (0, 255, 0)  # Green
    else:
        label_text = "Without Mask"
        color = (0, 0, 255)  # Red

    # Display the label and bounding box on the frame
    cv2.putText(frame, label_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)
    cv2.rectangle(frame, (10, 10), (frame.shape[1] - 10, frame.shape[0] - 10), color, 2)

    # Show the frame with the label
    cv2.imshow("Face Mask Detection", frame)

    # Break the loop when 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video stream and close OpenCV windows



In [5]:
vs.release()
cv2.destroyAllWindows()